# 🏥 Hospital Data Cleaning Project

**Goal**: Clean and prepare hospital admission data to enable accurate analysis of patient demographics, admission types, diagnoses, and healthcare payment trends.


## 📋 Project Outline
1. Load the dataset
2. Explore the structure and nulls
3. Drop or impute null values
4. Standardize and format text
5. Remove duplicates
6. Final check and export


## Explore the Data

In [ ]:
import pandas as pd

df = pd.read_csv('Hospital_Inpatient_Discharges__SPARCS_De-Identified___2021_20231012.csv')
df.shape
df.info()
df.head()
df.columns


C:\Users\kathir\AppData\Local\Temp\ipykernel_15916\1388219114.py:1: DtypeWarning: Columns (10,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Hospital_Inpatient_Discharges__SPARCS_De-Identified___2021_20231012.csv')


Index(['Hospital Service Area', 'Hospital County',
       'Operating Certificate Number', 'Permanent Facility Id',
       'Facility Name', 'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race',
       'Ethnicity', 'Length of Stay', 'Type of Admission',
       'Patient Disposition', 'Discharge Year', 'CCSR Diagnosis Code',
       'CCSR Diagnosis Description', 'CCSR Procedure Code',
       'CCSR Procedure Description', 'APR DRG Code', 'APR DRG Description',
       'APR MDC Code', 'APR MDC Description', 'APR Severity of Illness Code',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'APR Medical Surgical Description', 'Payment Typology 1',
       'Payment Typology 2', 'Payment Typology 3', 'Birth Weight',
       'Emergency Department Indicator', 'Total Charges', 'Total Costs'],
      dtype='object')

### Drop irrelevant columns

In [30]:
columns_to_drop = ['Operating Certificate Number','Hospital County' ]
df.drop(columns=columns_to_drop, inplace=True)

## Rename Columns for Clarity

In [31]:
df.rename(columns={
   'Zip Code - 3 digits' : 'zipcode3',
   'APR DRG Code' : 'DRG_code',
   'Age Group' : 'Age_group',   
}, inplace=True)

## handle missing values

In [32]:
df.isnull().sum()

Hospital Service Area                    10642
Permanent Facility Id                    10642
Facility Name                                0
Age_group                                    0
zipcode3                                 45062
Gender                                       0
Race                                         0
Ethnicity                                    0
Length of Stay                               0
Type of Admission                            0
Patient Disposition                          0
Discharge Year                               0
CCSR Diagnosis Code                       1634
CCSR Diagnosis Description                1634
CCSR Procedure Code                     576021
CCSR Procedure Description              576021
DRG_code                                     0
APR DRG Description                          0
APR MDC Code                                 0
APR MDC Description                          0
APR Severity of Illness Code                 0
APR Severity 

### 1️⃣ Dropping Columns with Too Many Nulls
- Dropping: `CCSR Procedure Code`, `Procedure Description`, `Birth Weight`, `Payment Typology 2`, `Payment Typology 3`, `zipcode3`
- Justification: Over 25% nulls — not recoverable or usable in modeling

In [33]:
df.drop(columns=[
    'Birth Weight', 'CCSR Procedure Description', 'CCSR Procedure Code', 'Payment Typology 2', 'Payment Typology 3', 'zipcode3'],
        inplace=True)


In [34]:
df.isnull().sum()

Hospital Service Area                  10642
Permanent Facility Id                  10642
Facility Name                              0
Age_group                                  0
Gender                                     0
Race                                       0
Ethnicity                                  0
Length of Stay                             0
Type of Admission                          0
Patient Disposition                        0
Discharge Year                             0
CCSR Diagnosis Code                     1634
CCSR Diagnosis Description              1634
DRG_code                                   0
APR DRG Description                        0
APR MDC Code                               0
APR MDC Description                        0
APR Severity of Illness Code               0
APR Severity of Illness Description     2550
APR Risk of Mortality                   2550
APR Medical Surgical Description           0
Payment Typology 1                         0
Emergency 

## Handle Moderate Null values### 2️⃣ Imputing Low-Null Fields
- `Hospital Service Area`, `Permanent Facility Id`, `APR Severity of Illness Description`: Impute as `'Unknown'`
- `Permanent Facility Id`: Impute with Mode value


In [ ]:
df['APR Severity of Illness Description']= df['APR Severity of Illness Description'].fillna('Unknown')
df['APR Risk of Mortality'].fillna('Unknown', inplace=True)
df['Hospital Service Area'].fillna('Unknown', inplace=True)
df['Permanent Facility Id'].fillna('1456.0', inplace=True) # fill with mode

## Remove Critical rows like diagnosis

In [53]:
df.dropna(subset=['CCSR Diagnosis Code', 'CCSR Diagnosis Description'], inplace=True)
df.isnull().sum()

Hospital Service Area                  0
Permanent Facility Id                  0
Facility Name                          0
Age_group                              0
Gender                                 0
Race                                   0
Ethnicity                              0
Length of Stay                         0
Type of Admission                      0
Patient Disposition                    0
Discharge Year                         0
CCSR Diagnosis Code                    0
CCSR Diagnosis Description             0
DRG_code                               0
APR DRG Description                    0
APR MDC Code                           0
APR MDC Description                    0
APR Severity of Illness Code           0
APR Severity of Illness Description    0
APR Risk of Mortality                  0
APR Medical Surgical Description       0
Payment Typology 1                     0
Emergency Department Indicator         0
Total Charges                          0
Total Costs     

## Standardize Column Names

In [55]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
df.columns

Index(['hospital_service_area', 'permanent_facility_id', 'facility_name',
       'age_group', 'gender', 'race', 'ethnicity', 'length_of_stay',
       'type_of_admission', 'patient_disposition', 'discharge_year',
       'ccsr_diagnosis_code', 'ccsr_diagnosis_description', 'drg_code',
       'apr_drg_description', 'apr_mdc_code', 'apr_mdc_description',
       'apr_severity_of_illness_code', 'apr_severity_of_illness_description',
       'apr_risk_of_mortality', 'apr_medical_surgical_description',
       'payment_typology_1', 'emergency_department_indicator', 'total_charges',
       'total_costs'],
      dtype='object')

## Fix Categorical Columns

In [ ]:
df['gender'] = df['gender'].str.upper().str.replace('M', 'MALE').str.replace('F', 'FEMALE')
df['gender'] = df['gender'].str.strip().str.upper().str.replace('U', 'UNKNOWN')
df['gender']

0            MALE
1          FEMALE
2          FEMALE
3            MALE
4          FEMALE
            ...  
2101583    FEMALE
2101584      MALE
2101585      MALE
2101586    FEMALE
2101587    FEMALE
Name: gender, Length: 2099954, dtype: object

In [74]:
df.groupby(['race', 'gender']).size()

race                    gender 
Black/African American  FEMALE     213780
                        MALE       172107
                        UNKNOWN        29
Multi-racial            FEMALE      12706
                        MALE         9848
                        UNKNOWN         3
Other Race              FEMALE     301808
                        MALE       248281
                        UNKNOWN        60
White                   FEMALE     616277
                        MALE       524991
                        UNKNOWN        64
dtype: int64

## Drop Duplicated Rows

In [ ]:
df.duplicated().sum()

np.int64(31580)

In [77]:
df = df.drop_duplicates()

In [78]:
df.duplicated().sum()

np.int64(0)

## Final Null Check

In [79]:
df.isnull().sum()

hospital_service_area                  0
permanent_facility_id                  0
facility_name                          0
age_group                              0
gender                                 0
race                                   0
ethnicity                              0
length_of_stay                         0
type_of_admission                      0
patient_disposition                    0
discharge_year                         0
ccsr_diagnosis_code                    0
ccsr_diagnosis_description             0
drg_code                               0
apr_drg_description                    0
apr_mdc_code                           0
apr_mdc_description                    0
apr_severity_of_illness_code           0
apr_severity_of_illness_description    0
apr_risk_of_mortality                  0
apr_medical_surgical_description       0
payment_typology_1                     0
emergency_department_indicator         0
total_charges                          0
total_costs     

## Final Duplicates Check

In [80]:
df.duplicated().sum()

np.int64(0)

In [82]:
df.shape

(2068374, 25)

## ✅ Final Summary

- Dropped 6 high-null columns
- Imputed 4 low-null columns
- Cleaned text and standardized formatting
- Removed duplicates
- Final row count: 20,68,374
- Final column Count: 25

